Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-11-07 16:44:01--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.128.149
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.128.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’

instacart_online_gr 100%[===================>] 196.03M  96.7MB/s    in 2.0s    

2019-11-07 16:44:03 (96.7 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [3]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01



## Join Data Practice

**These are the top 10 most frequently order products. How many time was each ordered?**

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

In [4]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


In [0]:
# Import Everything
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.options.display.max_rows=999
pd.options.display.max_columns=100

In [0]:
# Keeping my notebook DRY
def checkData(filename):
  return pd.read_csv(f'{filename}.csv').sample(5)

In [7]:
checkData('aisles')

,aisle_id,aisle
79,80,deodorants
116,117,nuts seeds dried fruit
16,17,baking ingredients
111,112,bread
59,60,trash bags liners


In [8]:
checkData('orders')

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
2813328,1778672,169760,prior,4,3,21,4.0
3185727,689180,192083,prior,19,2,12,6.0
269848,2340253,16324,prior,8,1,14,30.0
2248300,3295094,135400,prior,31,6,11,12.0
733993,1837698,44176,prior,13,3,13,6.0


In [9]:
checkData('departments')

,department_id,department
17,18,babies
18,19,snacks
13,14,breakfast
19,20,deli
2,3,bakery


In [10]:
checkData('order_products__train')

,order_id,product_id,add_to_cart_order,reordered
22603,54503,38383,14,0
352463,858316,22935,14,1
560256,1376927,47357,10,1
65975,162285,3363,4,0
646271,1590450,36076,33,0


In [11]:
checkData('order_products__prior')

,order_id,product_id,add_to_cart_order,reordered
3454817,365017,17830,7,0
13706894,1446772,8516,10,1
625049,66124,24184,14,1
29569984,3118556,4605,2,0
28147729,2968248,14474,6,0


In [12]:
checkData('products')

,product_id,product_name,aisle_id,department_id
46440,46441,Calendula Body Lotion,102,18
6222,6223,Original Almond Coconut Blend,91,16
3199,3200,Mint Coconut 3 In 1 Wash,25,11
6030,6031,Ultra Half & Half,53,16
6139,6140,Whole Bean Coffee French Roast,26,7


### 1. Write down which columns you need and which dataframes have them.

> product_name - products.csv

> product_id - order_products__prior

> order_id - order_products_prior

In [0]:
# Make a df I can call for the products csv
prod_df = pd.read_csv('products.csv')

In [0]:
# Same for order_products_prior csv
opp_df = pd.read_csv('order_products__prior.csv')

### 2. Merge these into a single dataframe.

In [15]:
# Merged the to dataframes together inner so it only keep the like values I don't think the added NaN will be helpful
fullord = pd.merge(opp_df, prod_df, on='product_id', how='inner')
fullord.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16


In [0]:
zoomFullOrd = fullord.drop(['add_to_cart_order', 'reordered', 'aisle_id', 'department_id'], axis=1)

### 3. Use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [17]:
# I need the 10 most frequently ordered products represented as n
n = 10
# On the product_name column give me all the unique(in order by most called) values up to the n(10)th one then display it in a list
zoomFullOrd['product_name'].value_counts()[:n].index.tolist()

['Banana',
 'Bag of Organic Bananas',
 'Organic Strawberries',
 'Organic Baby Spinach',
 'Organic Hass Avocado',
 'Organic Avocado',
 'Large Lemon',
 'Strawberries',
 'Limes',
 'Organic Whole Milk']

## Reshape Data Section
- Complete the code cells we skipped near the beginning of the notebook
- Table 2 --> Tidy
- Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

**1. Replicate the lesson code.**

In [18]:
# Replicated the lesson code.

myindex=['John Smith', 'Jane Doe', 'Mary Johnson']
mycolumns=['treatmenta', 'treatmentb']

table1 = pd.DataFrame(
            [[np.nan, 2],
             [16, 11],
             [3, 1]],
          columns=mycolumns,
        index=myindex
)
table1

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


In [19]:
table1 = table1.reset_index()
table1

,index,treatmenta,treatmentb
0,John Smith,NaN,2
1,Jane Doe,16.0,11
2,Mary Johnson,3.0,1


In [20]:
tidy = table1.melt(id_vars='index', value_vars=['treatmenta', 'treatmentb'])
tidy

,index,variable,value
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [21]:
tidy = tidy.rename(columns={'index':'Name', 'variable':'Trt', 'value':'Results'})
tidy

,Name,Trt,Results
0,John Smith,treatmenta,NaN
1,Jane Doe,treatmenta,16.0
2,Mary Johnson,treatmenta,3.0
3,John Smith,treatmentb,2.0
4,Jane Doe,treatmentb,11.0
5,Mary Johnson,treatmentb,1.0


In [32]:
tidy['Trt'] = tidy['Trt'].str.replace('treatmet', '')
tidy

,Name,Trt,Results
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0


In [33]:
table2 = table1.T
table2

,0,1,2
index,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16,3
treatmentb,2,11,1


**2. Tidy --> Table 2**

In [34]:
wayback = tidy.pivot(index='Name', columns='Trt', values='Results').T
wayback

Name,Jane Doe,John Smith,Mary Johnson
Trt,,,
a,16.0,NaN,3.0
b,11.0,2.0,1.0


### Flights dataset pivot table

**1. Load flights dataset**

In [0]:
flights = sns.load_dataset('flights')

In [37]:
flights.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


**2. Create pivots table showing the number of passengers by months and year. index = year | column = months**

In [40]:
liftoff = flights.pivot(index='year', columns='month', values='passengers')
liftoff

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####